# 1D spectroscopy

In [1]:
%gui qt

In [1]:
%matplotlib 

Using matplotlib backend: TkAgg


In [2]:
# import modules
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os.path
import numpy as np
import visa
import json
import myplots2, myutils # custom modules
import SIM              # instrument drivers
import AgilentN5230A
import LSCI332S
import HittiteHMC
import AgilentMXG
import time

# instrument initialization
pna = AgilentN5230A.AgilentN5230A(16)
#spec = HittiteHMC.HittiteHMC(18)
spec = HittiteHMC.HittiteHMC(10)
#spec = AgilentMXG.AgilentMXG(30)
#sim = SIM.SIM(19 , channel=3)
sim_glb = SIM.SIM(18, channel=3)
sim_chip = SIM.SIM(18, channel=1)
lsci = LSCI332S.LSCI332S(2)

In [55]:
reload(myplots2)

<module 'myplots2' from 'myplots2.py'>

In [26]:
volt_glb =0.055  #0.885 #14.645 #0.025
volt_chip = 0
sim_glb.setVoltage(volt_glb)
sim_chip.setVoltage(volt_chip)

In [29]:
################################################################################
# set up parameters
readoutFreq = 6.90276e9 #6.85835e9 #6.79573e9 #6.7947e9 #6.83444e9 #6.83614e9 #6.84248e9 #6.83759e9 #6.84371e9 
readoutPwr = -50
numPoints, IFbandwidth, avg = 50, 100, 1 
startFreq, endFreq, stepFreq = 3.6e9, 5e9, 5e6
specPwr = 0 # nominal power from mw source
fixedAtt = -40  # additional fixed attenuator if any
#volt_glb = 1.2 #0.388 #1.87 #1.95 #1.79
#volt_chip = 0 #-11.2  
sweepNumber = 1 # 1: one-way, 2: round-trip, 3: one-ways + roundtrip, 4: two roundtrip, ...

#save_path = 'Z:\User\JJ\sput\M5_1_spec'
#save_path = 'C:\Users\Lab User\Desktop\Albany1222'
#save_path = 'Z:\User\Jaseung\projects\CSFQ\W1\C5\spec'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W4\B5\BlueFors\Spec'
#surfix = 'Q1_Vglb={:.3g}_Vchip={:.3g}_LSS'.format(volt_glb, volt_chip)
surfix = 'Q4_Vglb={:.3g}_LSS'.format(volt_glb)
title = '1D spec' + ', ' + 'Vglb={} V'.format(volt_glb)  # figure title
##################################################################################

# dictionary for parameters to be saved in log file
specPwr_total = specPwr + fixedAtt
header = 'freq (Hz) mag(dB) phase(degree) real imag'
para = dict([('readout Freq (Hz)',readoutFreq),('readout Power (dBm)', readoutPwr),('numPoints',numPoints),
             ('IFbandwidth (Hz)', IFbandwidth),('avg',avg),('Spec freq start (Hz)',startFreq),('Spec freq end (Hz)',endFreq),
             ('Spec step Freq (Hz)',stepFreq),('Spec power(dBm)', specPwr_total),('volt_glb (V)',volt_glb), ('volt_chip (V)',volt_chip), ('header',header)])


# file setup
timeStr = time.strftime("%H%M%S")
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'Spec1D_fr{:.9g}_Pr{:.9g}_fs{:.9g}_{:.9g}_{:.9g}_Ps{:.9g}_V{:.3g}_T{:.3g}_{}_{}' 
baseFileName = baseStr.format(readoutFreq/1e9, readoutPwr, startFreq/1e9, endFreq/1e9, stepFreq/1e9,
                              specPwr_total, volt_glb, FAA, surfix, timeStr)
dataFileName  = os.path.join(save_path, baseFileName + '.dat')
logFileName = os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f_log:
    json.dump(para,f_log,indent=4, sort_keys=True)
f_data = open(dataFileName, 'a')
   
# plot setup
plotobj= myplots2.plot1D_2sub(xlabel1 = 'Spec. Frequency', xunit1='Hz', ylabel1='S21', yunit1='dB',  
                             xlabel2 = 'Spec Frequency', xunit2='Hz', ylabel2='Phase', yunit2='degree',
                             title = title, plotStyle='b-o')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('CW')
pna.setCWFreq(readoutFreq)
pna.setPwr(readoutPwr)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

spec.pwrOn()
spec.setPwr(specPwr)

#sim_glb.setVoltage(volt_glb)
#sim_chip.setVoltage(volt_chip)
time.sleep(2)

# data initialization
xdata, ydata1, ydata2 = np.array([]), np.array([]),np.array([])

# sweeper obj
sweeper = myutils.Sweeper1D(start=startFreq, end=endFreq, step=stepFreq,
                            sweepNum=sweepNumber)

################ sweep start ################
while not sweeper.stop(): 
    #clear_output()
    #print('fs={}'.format(sweeper.curr_val))
    
    # configure
    pna.avgClear()
    spec.setFreq(sweeper.curr_val)
    
    # fetch data from PNA
    freq, S21= pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    S21_mag_log_avg = np.average(S21_mag_log) 
    S21_phase_avg = np.average(S21_phase)
    
    # create spec freq, S21 1D array
    xdata = np.append(xdata, sweeper.curr_val)
    ydata1 = np.append(ydata1, S21_mag_log_avg)
    ydata2 = np.append(ydata2, S21_phase_avg)
    
    # plot and update
    plotobj.update(xdata, ydata1, xdata, ydata2)
    
    # save data in each iteration
    string = '{:.9g}\t{:.6g}\t{:.6g} \n'.format(sweeper.curr_val, S21_mag_log_avg,S21_phase_avg)
    f_data.write(string)
    
    # update sweep
    sweeper.update()
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
spec.pwrOff()
pna.setSweepType(mode='linear')

f_data.close()
f_log.close()
print('Measurement complete!!!')

Initialized S21 Measurement 'S21Meas'
Measurement complete!!!


# Spec vs Power, 2D

In [ ]:
%%time
####################################################################################################################
# set up parameters
readoutFreq = 6.95394e9
readoutPwr = -60
numPoints, IFbandwidth, avg = 50, 100, 1 
startFreq, endFreq, stepFreq = 4.35e9, 4.7e9, 2e6
specPwr = -10 # nominal power from mw source
fixedAtt = -20  # additional fixed attenuator if any
startPwr, endPwr, stepPwr = 20, 0, 1
surfix = 'Q5_Vglb={:.3g}'.format(volt_glb)
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017_2nd\Spec'
commentStr = ''
####################################################################################################################

# dictionary for parameters to be saved in log file.
specPwr_total = specPwr + fixedAtt
xID, xStart, xEnd, xStep = 'Power (dBm)', startPwr, endPwr, stepPwr
yID, yStart, yEnd, yStep = 'Spec. Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Spec power', specPwr_total)
             , ('readout Freq (Hz)',readoutFreq),('readout Power (dBm)', readoutPwr)
             , ('numPoints',numPoints), ('IFbandwidth(Hz)', IFbandwidth)
             , ('avg',avg),  ('comment', commentStr)])

# file setup
timeStr = time.strftime("%H%M%S")
FAA = float(lsci.tempGet('B'))  # get temperature
baseStr = 'SpecvsP_2D_fr{:.9g}_Pr{:.9g}_fs{:.9g}_{:.9g}_{:.9g}_Ps{:.9g}_P{:.3g}_{:.3g}_{:.3g}_T{:.3g}_{}_{}'
baseFileName = baseStr.format(readoutFreq/1e9, readoutPwr, startFreq/1e9, endFreq/1e9, stepFreq/1e9, specPwr_total,
                              startPwr, endPwr, stepPwr, FAA, surfix, timeStr)
dataFileName1  = os.path.join(save_path, baseFileName + '_mag.dat')
dataFileName2  = os.path.join(save_path, baseFileName + '_phase.dat')
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f_log:
    json.dump(para,f_log,indent=4, sort_keys=True)

f_log = open(logFileName, 'a') 

# plot setup
plotobj= myplots2.plot2D(xlabel1 = '', ylabel1='S21 (dB)', 
                             xlabel2 = 'Spec Frequency(Hz)', ylabel2='Phase(degree)',
                             xlabel3 = 'Volt (V)', ylabel3='Spec Frequency(Hz)',
                             xlabel4 = 'Volt (V)', ylabel4='Spec Frequency(Hz)',
                             title = '2D spectroscopy', figsize=(12,8))

# preconfigure instrument
pna.setupMeas()
pna.setSweepType(mode='CW')
pna.setCWFreq(readoutFreq)
pna.setPwr(readoutPwr)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

spec.setPwr(specPwr)
spec.pwrOn()

# data initialization
data1_2D, data2_2D = np.array([]), np.array([])

################ sweep start ################
#for specFreq in arange(startFreq, endFreq + stepFreq, stepFreq): # traverse in [pmin, pmax].
for volt in np.arange(startVolt, endVolt+stepVolt, stepVolt):
    #configure para
    sim_chip.setVoltage(volt)
    
    #readoutFreq = getBiasFreq(0) # find bias point from cavity resonance curve
    readoutFreq = getBiasFreq(0) # 0 is lower freq side, 3dB higher
        
    pna.setSweepType(mode='CW')
    pna.setCWFreq(readoutFreq)
    pna.setPwr(readoutPwr)
    pna.avgCount(avg)
    pna.numPoints(numPoints)
    pna.setIF(IFbandwidth)

    # data initialization
    xdata1, ydata1, ydata2 = np.array([]), np.array([]), np.array([])
    
    for specFreq in np.arange(startFreq, endFreq+stepFreq, stepFreq):
        # configure
        pna.avgClear()
        spec.setFreq(specFreq)
    
        # fetch data from PNA
        freq, S21= pna.getData(cplx=True) # S21 in linear scale
    
        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        S21_mag_log_avg = np.average(S21_mag_log) 
        S21_phase_avg = np.average(S21_phase)

        xdata1 = np.append(xdata1, specFreq)
        ydata1 = np.append(ydata1, S21_mag_log_avg)
        ydata2 = np.append(ydata2, S21_phase_avg)

        # plot and update
        plotobj.update1D(xdata1, ydata1, xdata1, ydata2)

    
    if data1_2D.size == 0: # when S21_mag_2D is empty
        data1_2D = ydata1 # make 2D array
        data1_2D.shape = (data1_2D.size, 1)
        data2_2D = ydata2
        data2_2D.shape = (data2_2D.size, 1)
        
    else:
        data1_2D = np.column_stack((data1_2D, ydata1))
        data2_2D = np.column_stack((data2_2D, ydata2))
             
    # plot and update
    extent = [startVolt, volt, startFreq, endFreq]
    plotobj.update2D(data1_2D, data2_2D, extent, cmap='Blues_r')
    
    # save data in each iteration
    f_mag = open(dataFileName1, 'w') 
    f_phase = open(dataFileName2, 'w') 
    np.savetxt(f_mag, data1_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, data2_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
     
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
pna.setSweepType(mode='linear')
spec.pwrOff()
sim.setVoltage(endV)
f_log.close()


## 2D spectroscopy

In [7]:
from scipy.optimize import curve_fit
# measure resonance dip and do fit. Get the frequency at 3dB points
def getBiasFreq(LorR):
    pna.setSweepType(mode='linear')
    pna.setFreqStartEnd(6.79e9, 6.81e9)
    pna.setPwr(-60)
    pna.avgCount(1)
    pna.numPoints(801)
    pna.setIF(50)
    pna.avgClear()
    
    freq, S21= pna.getData(cplx=True)
    S21_mag = np.sqrt(np.real(S21)**2 + np.imag(S21)**2)
    S21_mag_log = np.log10(S21_mag)
    
    f0 = freq[np.argmin(S21_mag_log)]
    
    #popt, pcov = doLorenzianFit(freq, S21_mag_log) # fit_result is tuple of f0,delF and Q
    #f0, delF = popt[0], popt[1]
    return f0
#     if LorR == 0:
#         return f0 - 30e3
#     else:
#         return f0 + 30e3

# def lorentzian(x, f0, delF, scaling):
#     return 

# def doLorenzianFit(freq, S21_mag_log):
#     # set initial value
#     p0 =[]
#     p0.append(freq(np.argmin(S21_mag_log))) # f0
    
#     popt, pcov = curve_fit(lorenzin, freq, S21_mag, p0)
#     return popt, pcov


In [10]:
%%time
####################################################################################################################
# set up parameters
readoutPwr = -60
readoutFreq = getBiasFreq(0)
numPoints, IFbandwidth, avg = 50, 100, 1
startFreq, endFreq, stepFreq = 2.8e9, 6.5e9, 5e6
specPwr = 0
startVolt, endVolt, stepVolt =-8.25, -5.25, 0.025
fixedAtt = -20
endV = endVolt

surfix = 'Q1_Vglb={:.3g}'.format(volt_glb)
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017_2nd\Spec'
commentStr = ''
####################################################################################################################

# dictionary for parameters to be saved in log file.
specPwr_total = specPwr + fixedAtt
xID, xStart, xEnd, xStep = 'Voltage (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Spec. Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Spec power', specPwr_total)
             , ('readout Freq (Hz)',readoutFreq),('readout Power (dBm)', readoutPwr)
             , ('numPoints',numPoints), ('IFbandwidth(Hz)', IFbandwidth)
             , ('avg',avg),  ('comment', commentStr)])

# file setup
timeStr = time.strftime("%H%M%S")
FAA = float(lsci.tempGet('B'))  # get temperature
baseStr = 'Spec2D_fr{:.9g}_Pr{:.9g}_fs{:.9g}_{:.9g}_{:.9g}_Ps{:.9g}_V{:.6g}_{:.3g}_{:.3g}_T{:.3g}_{}_{}'
baseFileName = baseStr.format(readoutFreq/1e9, readoutPwr, startFreq/1e9, endFreq/1e9, stepFreq/1e9, specPwr_total,
                              startVolt, endVolt, stepVolt, FAA, surfix, timeStr)
dataFileName1  = os.path.join(save_path, baseFileName + '_mag.dat')
dataFileName2  = os.path.join(save_path, baseFileName + '_phase.dat')
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f_log:
    json.dump(para,f_log,indent=4, sort_keys=True)

f_log = open(logFileName, 'a') 


# plot setup
plotobj= myplots.plot2D_4sub(xlabel1 = '', ylabel1='S21 (dB)', 
                             xlabel2 = 'Spec Frequency(Hz)', ylabel2='Phase(degree)',
                             xlabel3 = 'Volt (V)', ylabel3='Spec Frequency(Hz)',
                             xlabel4 = 'Volt (V)', ylabel4='Spec Frequency(Hz)',
                             title = '2D spectroscopy', figsize=(12,8))

# preconfigure instrument
pna.setupMeas()
pna.setSweepType(mode='CW')
pna.setCWFreq(readoutFreq)
pna.setPwr(readoutPwr)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

spec.setPwr(specPwr)
spec.pwrOn()

# data initialization
data1_2D, data2_2D = np.array([]), np.array([])

################ sweep start ################
#for specFreq in arange(startFreq, endFreq + stepFreq, stepFreq): # traverse in [pmin, pmax].
for volt in np.arange(startVolt, endVolt+stepVolt, stepVolt):
    #configure para
    sim_chip.setVoltage(volt)
    
    #readoutFreq = getBiasFreq(0) # find bias point from cavity resonance curve
    readoutFreq = getBiasFreq(0) # 0 is lower freq side, 3dB higher
        
    pna.setSweepType(mode='CW')
    pna.setCWFreq(readoutFreq)
    pna.setPwr(readoutPwr)
    pna.avgCount(avg)
    pna.numPoints(numPoints)
    pna.setIF(IFbandwidth)

    # data initialization
    xdata1, ydata1, ydata2 = np.array([]), np.array([]), np.array([])
    
    for specFreq in np.arange(startFreq, endFreq+stepFreq, stepFreq):
        # configure
        pna.avgClear()
        spec.setFreq(specFreq)
    
        # fetch data from PNA
        freq, S21= pna.getData(cplx=True) # S21 in linear scale
    
        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        S21_mag_log_avg = np.average(S21_mag_log) 
        S21_phase_avg = np.average(S21_phase)

        xdata1 = np.append(xdata1, specFreq)
        ydata1 = np.append(ydata1, S21_mag_log_avg)
        ydata2 = np.append(ydata2, S21_phase_avg)

        # plot and update
        plotobj.update1D(xdata1, ydata1, xdata1, ydata2)

    
    if data1_2D.size == 0: # when S21_mag_2D is empty
        data1_2D = ydata1 # make 2D array
        data1_2D.shape = (data1_2D.size, 1)
        data2_2D = ydata2
        data2_2D.shape = (data2_2D.size, 1)
        
    else:
        data1_2D = np.column_stack((data1_2D, ydata1))
        data2_2D = np.column_stack((data2_2D, ydata2))
             
    # plot and update
    extent = [startVolt, volt, startFreq, endFreq]
    plotobj.update2D(data1_2D, data2_2D, extent, cmap='Blues_r')
    
    # save data in each iteration
    f_mag = open(dataFileName1, 'w') 
    f_phase = open(dataFileName2, 'w') 
    np.savetxt(f_mag, data1_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, data2_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
     
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
pna.setSweepType(mode='linear')
spec.pwrOff()
sim.setVoltage(endV)
f_log.close()


Initialized S21 Measurement 'S21Meas'


C:\Anaconda\lib\site-packages\matplotlib\axes\_base.py:2562: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-8.25, right=-8.25
  'left=%s, right=%s') % (left, right))


KeyboardInterrupt: 

# S21 continuous in CW vs Time

In [130]:
##################################################################################################
# set up parameters
readoutFreq = 7.062425e9
readoutPwr = -70
numPoints, IFbandwidth, avg = 10, 1000, 2
#specFreq = 5e9
#specPwr = -20
volt = 0
###################################################################################################

# plot setup
plotobj= myplots.plot1D_2sub(xlabel1='index', ylabel1='S21 (dB)', ylabel2='Phase(degree)', title = 'CW mode')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('CW')
pna.setCWFreq(readoutFreq)
pna.setPwr(readoutPwr)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()
pna.avgClear()
#spec.pwrOn()
#spec.setPwr(specPwr)

sim.setVoltage(volt)

# data initialization
xdata, ydata1, ydata2 = np.array([]), np.array([]),np.array([])

################ sweep start ################

N =1000
for i in range(N): 
    i +=1
    # configure
    #spec.setFreq(sweeper.curr_val)
    
    # fetch data from PNA
    freq, S21= pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    S21_mag_log_avg = np.average(S21_mag_log) 
    S21_phase_avg = np.average(S21_phase)
    # create spec freq, S21 1D array
    xdata = np.append(xdata, i)
    ydata1 = np.append(ydata1, S21_mag_log_avg)
    ydata2 = np.append(ydata2, S21_phase_avg)
    
    # plot and update
    plotobj.update(xdata, ydata1, xdata, ydata2)
    
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
#spec.pwrOff()
pna.setSweepType(mode='linear')


Initialized S21 Measurement 'S21Meas'


KeyboardInterrupt: 

In [125]:
import LSCI332S
lsci = LSCI332S.LSCI332S(2)

In [126]:
FAA = lsci.tempGet('B')

+00.000
